Итак, задача — сегментация текста с расстановкой пробелов в предложениях.

Я решил использовать цепи Маркова и берт, предобученный на датасетах объявлений Авито, выложенных на кеггле вот тут — https://www.kaggle.com/competitions/avito-category-prediction/data.

In [1]:
! pip install nltk -q
! pip install pandas -q # на всякий случай

## Markov Chains

### Делаем словари биграмм и униграмм на нашем корпусе

In [9]:
import nltk
from collections import Counter
import re
import csv
import pandas as pd

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [10]:
def preprocess(text):
    if not isinstance(text, str):   # проверяет и если не строка
        text = str(text)  # превращает в строку
    text = text.lower()
    # будем учитывать также знаки препинания
    text = re.sub(r'[^а-яёa-z0-9,.!?;:\-—\s]', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [11]:
def build_and_save_freqs(texts, unigram_file='unigram.csv', bigram_file='bigram.csv'):
    unigram_counts = Counter()
    bigram_counts = Counter()

    for text in texts:
        text = preprocess(text)
        tokens = nltk.word_tokenize(text)
        unigram_counts.update(tokens)
        bigram_counts.update(nltk.bigrams(tokens))

    # сохраняем словарь униграмм
    with open(unigram_file, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['word', 'count'])  # заголовок
        for word, count in unigram_counts.items():
            writer.writerow([word, count])

    # сохраняем словарь биграмм
    with open(bigram_file, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['word1', 'word2', 'count'])  # заголовок
        for (w1, w2), count in bigram_counts.items():
            writer.writerow([w1, w2, count])

In [15]:
# читаем корпус (описания товаров с авито)
corpora = pd.read_csv('test.csv')
corpora.head()

,title,description,itemid
0,Мастерка,Мастерка фирмы форвард. Белого цвета. В идеаль...,1778449823
1,Зимние сапоги,"Продаю зимние сапоги, в хорошем состоянии, все...",1677656962
2,Видеонаблюдение 8 камер,В комплект Atis AMD-2MIR-8kit входит: /\n1. Ку...,1758182804
3,Запчасти для GLE,Запчасти GLE,1689811299
4,Бластер nerf,Состояние 5+/\nПродаю потому что не нужен/\n18...,1804706240


In [16]:
# сохраняем словари биграмм и униграмм
build_and_save_freqs(corpora['description'])

### Подсчитываем вероятности на датасете

In [17]:
! pip install razdel -q

In [18]:
import math
import csv
import pandas as pd
import re
from razdel import tokenize  #  токенизатор от создателей natasha

In [19]:
def load_unigram_freq(file='unigram.csv'): # достаём униграммы
    unigram_freq = {}
    with open(file, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            unigram_freq[row['word']] = int(row['count'])
    return unigram_freq


def load_bigram_freq(file='bigram.csv'): # и биграммы
    bigram_freq = {}
    with open(file, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            bigram_freq[(row['word1'], row['word2'])] = int(row['count'])
    return bigram_freq


unigram_freq = load_unigram_freq()
bigram_freq = load_bigram_freq()

total_unigrams = sum(unigram_freq.values())
log_total_unigrams = math.log(total_unigrams)

total_bigrams = sum(bigram_freq.values())
log_total_bigrams = math.log(total_bigrams)

In [20]:
def unigram_log_prob(word):
    return math.log(unigram_freq.get(word, 1)) - log_total_unigrams


def bigram_log_prob(prev, curr):
    count_bigram = bigram_freq.get((prev, curr), 0)
    count_prev = unigram_freq.get(prev, 0)
    V = len(unigram_freq)
    return math.log((count_bigram + 1) / (count_prev + V))


def restore_spaces_bigram(text):
    n = len(text)
    max_word_len = max(len(w) for w in unigram_freq)
    dp = [{} for _ in range(n+1)]
    dp[0]["<s>"] = (0.0, -1, None)

    for i in range(1, n+1):
        for j in range(max(0, i - max_word_len), i):
            word = text[j:i]
            if word not in unigram_freq:
                continue
            for prev_word in dp[j]:
                prev_prob, _, _ = dp[j][prev_word]
                prob = prev_prob + bigram_log_prob(prev_word, word)
                if word not in dp[i] or prob > dp[i][word][0]:
                    dp[i][word] = (prob, j, prev_word)

    if not dp[n]:
        return []

    # считаем наиболее вероятное слово
    best_end_word = max(dp[n], key=lambda w: dp[n][w][0])
    words = []
    cur_pos = n
    cur_word = best_end_word
    while cur_pos > 0:
        words.append(cur_word)
        _, prev_pos, prev_word = dp[cur_pos][cur_word]
        cur_pos = prev_pos
        cur_word = prev_word

    words.reverse()
    return words

In [21]:
def preprocess(text):
    # токенизируем с помощью razdel (чуть лучше регулярок)
    tokens = list(tokenize(text))
    result = []

    punctuation = set([",", ".", "!", "?", ";", ":"])

    for token_obj in tokens:
        segment = token_obj.text

        if segment in punctuation and result:
            # если знак препинания — приклеиваем к предыдущему слову без пробела
            result[-1] += segment

        elif re.fullmatch(r'[а-яёa-z0-9]+', segment, flags=re.IGNORECASE):
            # если русский или английский непрерывный текст (без знаков препинания)
            words = restore_spaces_bigram(segment)
            if words:
                combined = " ".join(words)
                result.append(combined)
            else:
                result.append(segment)
        else:
            # остальное просто добавляем
            result.append(segment)

    return " ".join(result)

In [23]:
input_df = pd.read_csv("filename.txt", sep='^([^,]+),', engine='python')

output_data = []

# предсказывает пробелы
for idx, row in input_df.iterrows():
  text_no_spaces = row['text_no_spaces']
  predicted_text = preprocess(text_no_spaces)
  output_data.append({'id': row['id'], 'predicted': predicted_text})

output_df = pd.DataFrame(output_data)
output_df.to_csv('predicted_data.csv', index=False, encoding='utf-8')

In [24]:
output_df.head()

,id,predicted
0,0,куплю айфон 14 про
1,1,ищудомвПодмосковье
2,2,сдаюквартиру с мебелью и техникой
3,3,новый диван доставка недорого
4,4,отдам даром кошку


In [27]:
# создаём итоговый файл с позициями пробелов
def find_space_positions(original_text):
    positions = []
    idx_no_space = 0

    for ch in original_text:
        if ch == ' ':
            positions.append(idx_no_space)
        else:
            idx_no_space += 1

    return positions


df = pd.read_csv('predicted_data.csv')
df['predicted_positions'] = df['predicted'].apply(find_space_positions)

new_df = df[['id', 'predicted_positions']]
new_df.to_csv('predicted_positions.csv', index=False)

In [28]:
new_df.head()

,id,predicted_positions
0,0,"[5, 10, 12]"
1,1,[]
2,2,"[12, 13, 20, 21]"
3,3,"[5, 10, 18]"
4,4,"[5, 10]"


как итог — F1 53.985%

неплохо!

датасет нужно поискать получше: сейчас очень тяжёлый и охватывает не все темы (я использовал данные только test.csv, но можно смёрджить с train.csv, отфильтровать ненужное и взять какой-нибудь ещё датасет отвлечённых текстов — в тесте помимо выдержек из объявлений есть строчки из песен, которые сосем плохо парсятся)



## BERT

### Создание обучающего датасета

In [14]:
import pandas as pd
import re

import nltk
nltk.download('punkt_tab')

from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [2]:
df = pd.read_csv('test.csv', usecols=['description']).dropna()

In [15]:
def clean_text(text):
    text = text.replace('\xa0', ' ')
    text = re.sub(r'[\n\\/]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^а-яА-Яa-zA-Z0-9.,!?:;()\-–—\s]', '', text)
    text = text.strip()
    return text


def create_labels(original_text):
    no_space_text = original_text.replace(" ", "")
    labels = [0] * len(no_space_text)
    idx_no_space = 0
    for ch in original_text:
        if ch == " ":
            labels[idx_no_space - 1] = 1  # Ставим 1 после предыдущего символа без пробела
        else:
            idx_no_space += 1
    return no_space_text, labels


texts_no_spaces = []
labels_list = []

for text in df['description']:
    sentences = sent_tokenize(text, language='russian')
    for sent in sentences:
        no_space, labels = create_labels(clean_text(sent))
        texts_no_spaces.append(no_space)
        labels_list.append(labels)

train_df = pd.DataFrame({
    'text_no_spaces': texts_no_spaces,
    'labels': labels_list
})

In [16]:
train_df.to_csv('train.csv', index=False)

### Обучение модели

используем компактный rubert-tiny2

In [17]:
! pip install transformers datasets aiohttp -q

In [42]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
import ast

# компактный rubert-tiny2
MODEL_NAME = "cointegrated/rubert-tiny2"

df = pd.read_csv("train.csv")

# препобразуем строки из csv в списки
df['labels'] = df['labels'].apply(ast.literal_eval)

# инициализируем токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=2)


def tokenize_and_align_labels(examples):
    # examples приходят батчем (каждый элемент — список/массива значений)
    texts_raw = examples["text_no_spaces"]
    labels_raw = examples["labels"]

    texts = []
    labels = []

    # нормализация батча: приводим каждый текст к str (или объединяем список символов)
    for t, l in zip(texts_raw, labels_raw):
        # нормализуем текст в строку
        if t is None or (isinstance(t, float) and np.isnan(t)):
            text = "" # пустой текст, если NaN
        elif isinstance(t, (list, tuple)):  # если уже список символов
            text = "".join(map(str, t))
        else:
            text = str(t) # обычный str / numpy.str_
        texts.append(text)

        # нормализуем метки
        if isinstance(l, str):
            # на случай, если метки в csv всё ещё в виде строки "[0,1,...]"
            lab = ast.literal_eval(l)
        else:
            lab = l
        labels.append(list(lab))

    # токенизируем батч, получая offset_mapping
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_offsets_mapping=True
    )

    new_labels = []
    for i, offsets in enumerate(tokenized["offset_mapping"]):
        char_labels = labels[i]
        label_ids = []
        for (start, end) in offsets:
            if start == end:
                # special tokens / padding
                label_ids.append(-100)
            else:
                # берём метку для первого символа, покрываемого токеном
                if start < len(char_labels):
                    label_ids.append(char_labels[start])
                else:
                    # на случай, если токен выходит за длину разметки
                    label_ids.append(-100)
        new_labels.append(label_ids)

    tokenized["labels"] = new_labels
    tokenized.pop("offset_mapping")  # не нужен дальше
    return tokenized


dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

training_args = TrainingArguments(
    output_dir="./model_out",
    num_train_epochs=2, # сколько раз пройти весь тренировочный датасет
    per_device_train_batch_size=16, # размер батча на устройство
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    eval_strategy="no",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

# сохраняем модель
trainer.save_model("./model_out")
tokenizer.save_pretrained("./model_out")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3190 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,0.350700
100,0.262000
150,0.225800
200,0.210100
250,0.188200
300,0.175300
350,0.187700
400,0.181700
450,0.166200
500,0.165500


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


('./model_out/tokenizer_config.json',
 './model_out/special_tokens_map.json',
 './model_out/vocab.txt',
 './model_out/added_tokens.json',
 './model_out/tokenizer.json')

In [44]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

MODEL_NAME = "./model_out"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)
model.eval()

# загрузка тестовых данных
test_df = pd.read_csv("filename.txt", sep='^([^,]+),', engine='python')

def predict_positions(text_no_space):
    inputs = tokenizer(
        text_no_space,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128,
        return_offsets_mapping=True
    )

    offset_mapping = inputs.pop("offset_mapping")
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)[0].tolist()

    # Находим индексы символов, где предсказано 1
    positions = []
    for idx, (pred, (start, end)) in enumerate(zip(predictions, offset_mapping[0].tolist())):
        if start == end:  # спецтокены/паддинг
            continue
        if pred == 1 and start < len(text_no_space):
            positions.append(start)  # индекс символа, где должен быть пробел

    return str(positions)

test_df["predicted_positions"] = test_df["text_no_spaces"].apply(predict_positions)

test_df[["id", "predicted_positions"]].to_csv("predicted_positions.csv", index=False)

Кажется, датасет всё-таки неоптимальный :—(

Показал F1 7%
Попытался обновить тренировочный (убрав ограничение в 1000 дескрипшонов), но не успело посчитаться